## Artificial Neural Network approximations of Cauchy Inverse problem for linear PDEs
---

## Imports

In [1]:
from ANNPDE.PDE import ReverseChauchyPDE
from ANNPDE.PDE.shapes import (
    ElipseShape, 
    CircleShape, 
    LineShape
)
from ANNPDE.ANN import (
    LSTM,
    prepare_data,
    laplacian
)
import plotly.graph_objs as go
from random import randint
from tqdm import tqdm
from torch import nn
import pandas as pd
import numpy as np
import torch

## Setting Seed

In [2]:
SEED = randint(1, 1000000)
torch.manual_seed(SEED)
print('Seed:', SEED)

Seed: 756894


### Reverse Cauchy Problem

In [3]:
F_EXPR = 'E ** x1 * sin(x2) * cos(t)'
G_EXPR = ['E ** x1 * sin(x2)', 'E ** x1 * cos(x2)']
H_EXPR = 'E ** x1 * sin(x2)'
Xs_symbol, t_symbol = ['x1', 'x2'], 't'

print(
    '\n • ∂u(X, t)/∂t + Δu(X, t) = 0'
    '\n • u(X, t) = f =', F_EXPR,
    '\n • ∂u(X, t)/∂n = g =', G_EXPR,
    '\n • u(X, 0)  = h =', H_EXPR,
)


 • ∂u(X, t)/∂t + Δu(X, t) = 0
 • u(X, t) = f = E ** x1 * sin(x2) * cos(t) 
 • ∂u(X, t)/∂n = g = ['E ** x1 * sin(x2)', 'E ** x1 * cos(x2)'] 
 • u(X, 0)  = h = E ** x1 * sin(x2)


## Training parameters

In [4]:
T_SAMPLE = 128
E_SAMPLE = 512
D_SAMPLE = 2048
CENTER = np.array([0, 0])
RADIUS = 0.5
EDGE_CUT = [(0, np.pi * .5), (np.pi * 1, np.pi * 1.5)]

input_size = len(CENTER) + 1  # Input is (x1, x2, t)
hidden_layer_sizes = [10, 20, 50, 70, 40, 20]  # Customize your hidden layer sizes
batch_size_divider = 128
num_epochs = 100

ec_lambda = lambda x: "\n    ".join(
    [str(list(map(lambda x: round(x, 5), cut))) for cut in x]
)

print(
    '\nTraining parameters:'
    '\n • Time sample:', T_SAMPLE,
    '\n • Edge sample:', E_SAMPLE,
    '\n • Domain sample:', D_SAMPLE,
    '\n\n • Circle Center:', CENTER,
    '\n • Circle Radius:', RADIUS,
    f'\n • Circle Edge Cut(s): [\n    {ec_lambda(EDGE_CUT)}\n  ]',
    '\n\n • Hidden layer sizes:', hidden_layer_sizes,
    '\n • Batch Size Divider:',batch_size_divider, 
    '\n • Batch size :', ((E_SAMPLE + D_SAMPLE) * T_SAMPLE) // batch_size_divider,
    '=  (E_SAMPLE+D_SAMPLE)*T_SAMPLE)//batch_size_divider',
    '\n • Number of epochs:', num_epochs
)


Training parameters:
 • Time sample: 128 
 • Edge sample: 512 
 • Domain sample: 2048 

 • Circle Center: [0 0] 
 • Circle Radius: 0.5 
 • Circle Edge Cut(s): [
    [0, 1.5708]
    [3.14159, 4.71239]
  ] 

 • Hidden layer sizes: [10, 20, 50, 70, 40, 20] 
 • Batch Size Divider: 128 
 • Batch size : 2560 =  (E_SAMPLE+D_SAMPLE)*T_SAMPLE)//batch_size_divider 
 • Number of epochs: 100


## Time sampling

In [5]:
time = LineShape(
    seed=SEED,
    n=T_SAMPLE,
    start_point=0,
    end_point=np.pi/2,
    cross_sample_generate=1,
    even_sample=True
)
time_sample = time.get()

time.plot(3)

## Shape sampling

In [6]:
shape = CircleShape(
    seed=SEED,
    edge_n=E_SAMPLE,
    domain_n=D_SAMPLE,
    center=CENTER,
    radius=RADIUS,
    cross_sample_generate=1,
    edge_cuts_angle=EDGE_CUT,
    even_sample=True
)
edge_sample, domain_sample = shape.get()

shape.plot(1.3)

## Defining LSTM Model

In [7]:

model = LSTM(input_size, hidden_layer_sizes)

criterion = nn.MSELoss()
pde = ReverseChauchyPDE(
    f_function=F_EXPR,
    g_function=G_EXPR,
    h_function=H_EXPR,
    x_symbols=Xs_symbol,
    time_symbol=t_symbol,
    criterion=criterion
)
optimizer = torch.optim.Adam(model.parameters())

domain_input = torch.from_numpy(
    prepare_data(time_sample, domain_sample)
).float()
edge_input = torch.from_numpy(
    prepare_data(time_sample, edge_sample)
).float()

batch_size_edge = int(edge_input.shape[0] / batch_size_divider)
batch_size_domain = int(domain_input.shape[0] / batch_size_divider)

print(
    '\nSampling Information:'
    '\n • Domain input shape:', tuple(domain_input.shape),
    '\n • Edge input shape:', tuple(edge_input.shape),
    '\n • Domain Batch size:', batch_size_domain,
    '\n • Edge Batch size:', batch_size_edge
)


Sampling Information:
 • Domain input shape: (262144, 3) 
 • Edge input shape: (65536, 3) 
 • Domain Batch size: 2048 
 • Edge Batch size: 512


## Training Loop

In [8]:
tracking = {
    'epoch':      [],
    
    'tr_loss':    [],
    'f_loss':     [],
    'g_loss':     [],
    'h_loss':     [],

    'total_loss': [],
}

for epoch in range(num_epochs):

    domain_input = domain_input[torch.randperm(domain_input.size()[0])]
    edge_input = edge_input[torch.randperm(edge_input.size()[0])]   

    for key, value in tracking.items():
        if key == 'epoch':
            value.append(epoch + 1)
            continue
        value.append(0.0)

    print(f'\nEpoch {epoch+1}/{num_epochs} loop ...')
    
    for i in tqdm(range(batch_size_divider)):

        batch_domain = domain_input[i*batch_size_domain:(i+1)*batch_size_domain, :]
        batch_edge = edge_input[i*batch_size_edge:(i+1)*batch_size_edge, :]

        inputs = torch.cat((batch_domain, batch_edge), dim=0)
        inputs.requires_grad_(True)
        outputs = model(inputs)

        # Calculating the gradients

        gradients = torch.autograd.grad(
            outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True
        )[0]
        laplacians = laplacian(inputs, gradients)

        # Calculate Loss
        tr_loss, f_loss, g_loss, h_loss = pde.loss_function(
            inputs,
            batch_domain.size(0),
            outputs,
            gradients,
            laplacians,
            model,
        )

        combined_loss = tr_loss / inputs.size(0) + \
            f_loss / batch_edge.size(0) + \
                g_loss / batch_edge.size(0) + \
                    h_loss / inputs.size(0)
                    
        tracking['total_loss'][-1] += combined_loss.item()
        
        # Backward and optimize
        combined_loss.backward()
        optimizer.step()

        tracking['tr_loss'][-1] += tr_loss.item()
        tracking['f_loss'][-1] += f_loss.item()
        tracking['g_loss'][-1] += g_loss.item()
        tracking['h_loss'][-1] += h_loss.item()
    
    print(
        'Epoch [{}/{}], Total Loss: {:.4f}\nTR Loss: {:.4f}, F Loss: {:.4f}, ' \
        'G Loss: {:.4f}, H Loss: {:.4f}'.format(
            epoch + 1, 
            num_epochs, 
            tracking['total_loss'][-1] / batch_size_divider,
            tracking['tr_loss'][-1] / batch_size_divider, 
            tracking['f_loss'][-1] / batch_size_divider, 
            tracking['g_loss'][-1] / batch_size_divider, 
            tracking['h_loss'][-1] / batch_size_divider
        )
    ) 



Epoch 1/100 loop ...


  2%|▏         | 2/128 [03:03<3:23:14, 96.78s/it]

## Plotting progress

In [ ]:
tracking = pd.DataFrame(tracking)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=tracking['epoch'], y=tracking['tr_loss'],
    mode='lines+markers',
    name='TR Loss'
))
fig.add_trace(go.Scatter(
    x=tracking['epoch'], y=tracking['f_loss'],
    mode='lines+markers',
    name='F Loss'
))
fig.add_trace(go.Scatter(
    x=tracking['epoch'], y=tracking['g_loss'],
    mode='lines+markers',
    name='G Loss'
))
fig.add_trace(go.Scatter(
    x=tracking['epoch'], y=tracking['h_loss'],
    mode='lines+markers',
    name='H Loss'
))
fig.add_trace(go.Scatter(
    x=tracking['epoch'], y=tracking['total_loss'],
    mode='lines+markers',
    name='Total Loss'
))
fig.update_layout(
    title='Losses',
    xaxis_title='Epoch',
    yaxis_title='Loss',
    legend_title='Losses',
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
fig.show()

## Plotting defaults

In [ ]:
Ts = [np.pi/5, 3*np.pi/10, 2*np.pi/5, np.pi/2]
precision = 100
x_lin, y_lin, mesh_x, mesh_y, mask = shape.mesh(precision)
stacked_grid = np.stack((mesh_x.flatten(), mesh_y.flatten()), axis=-1, dtype=np.float32)

## Plotting output value

In [ ]:
with torch.no_grad():
    for i in range(len(Ts)):
        mesh_z = np.where(mask, 1, np.nan)
        for x in range(precision):
            for y in range(precision):
                if mesh_z[x, y] != 1:
                    continue
                input_tensor = torch.tensor(np.array(
                    np.array([x, y, Ts[i]]).reshape(1, 3)
                ), dtype=torch.float32)

                mesh_z[x, y] = model(input_tensor).squeeze()
        
        # Create the contour plot
        contour = go.Contour(
            x=x_lin,
            y=y_lin,
            z=mesh_z,
            colorscale='RdBu',
            ncontours=300,
            showscale=True,
            line=dict(width=0)
        )


        # Define layout to maintain aspect ratio
        layout = go.Layout(
            xaxis=dict(scaleanchor="y", scaleratio=1),
            yaxis=dict(scaleanchor="x", scaleratio=1),
            title=f"T = {Ts[i]}"
        )

        # Create the figure
        fig = go.Figure(data=[contour], layout=layout)

        # Show the plot
        fig.show()